In [10]:
import json
import requests

In [4]:
# Define your API keys
primary_key = "fc5281fc3a304be4a5b20017cd59cfac"
secondary_key = "82156f95fc494d59a53438144f59d883"

In [5]:
# Set up the headers with your API keys
headers = {
    "Ocp-Apim-Subscription-Key": primary_key,
    "Ocp-Apim-Subscription-Key-Secondary": secondary_key
}

In [6]:
# Define the API endpoint for retrieving API information
url_info = "https://api.artdatabanken.se/data-stewardship-api/v1/api_info"

In [7]:
# Send a GET request to the API endpoint with the headers
response = requests.get(url_info, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the API information from the response
    api_info = response.json()
    
    # Print all keys and their corresponding values
    print("API Information:")
    for key, value in api_info.items():
        print(f"{key}: {value}")
else:
    print("Failed to retrieve API information. Status code:", response.status_code)



API Information:
apiName: Nature data - systematic species observations
apiVersion: 1.0.5
apiReleased: 2023-09-20T11:25:10+02:00
apiDocumentation: https://github.com/biodiversitydata-se/SOS/blob/master/Docs/DataStewardship/README.md
apiChangelog: https://github.com/biodiversitydata-se/SOS/blob/master/Docs/DataStewardship/CHANGELOG.md
apiStatus: active


# Starts Csv aynchronous file order. 
When the file is ready, you will receive an email containing a download link. 
The limit is 2 million observations.

https://github.com/biodiversitydata-se/SOS/blob/master/Docs/Exports.md

In [8]:
# Define the endpoint URL
url_order = "https://api.artdatabanken.se/Exports/Order/Csv"

# Define the parameters for the file order
params = {
    "description": "Your download description",
    "cultureCode": "en-US",  # or any other appropriate culture code
    "outputFieldSet": "default",  # or specify the observation property field set
    "propertyLabelType": "PropertyName",  # or any other label type
    "flat": True,  # or False if you want hierarchical JSON structure
    "excludeNullValues": True  # or False if you want to include properties with null values
}

# Start the asynchronous file order
response = requests.post(url_order, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    print("File order started successfully. Check your email for the download link.")
else:
    print("Failed to start the file order. Status code:", response.status_code)


Failed to start the file order. Status code: 404


In [11]:
def fetch_species_observations(start_index, count):
    url = "https://sosgeo.artdata.slu.se/geoserver/wfs"
    params = {
        "REQUEST": "GetFeature",
        "OUTPUTFORMAT": "application/json",
        "VERSION": "2.0.0",
        "TYPENAMES": "SOS:SpeciesObservations",
        "COUNT": count,
        "SERVICE": "wfs",
        "CQL_FILTER": "isRedlisted='TRUE'",
        "STARTINDEX": start_index
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data. Status code:", response.status_code)
        return None

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Example usage
start_index = 0
count = 5000
observations = fetch_species_observations(start_index, count)
if observations:
    print("Retrieved", len(observations["features"]), "observations.")
    save_to_json(observations, "observations.json")
    print("Data saved to observations.json file.")



Retrieved 5000 observations.
Data saved to observations.json file.


In [14]:

# Load the JSON data from a file or API response
with open("observations.json", "r") as f:
    data = json.load(f)

# Iterate over the first 10 features
for feature in data["features"][:3]:
    # Print the occurrence ID, scientific name, and coordinates of the current feature
    print("Occurrence ID:", feature["properties"]["occurrenceId"])
    print("Scientific Name:", feature["properties"]["scientificName"])
    print("Coordinates:", feature["geometry"]["coordinates"])
    print()


Occurrence ID: urn:lsid:artportalen.se:sighting:115039001
Scientific Name: Sturnus vulgaris
Coordinates: [11.906710950209355, 57.64625498641013]

Occurrence ID: urn:lsid:artportalen.se:sighting:115040926
Scientific Name: Dryobates minor
Coordinates: [13.67667053766289, 59.741965223743584]

Occurrence ID: urn:lsid:artportalen.se:sighting:115039662
Scientific Name: Sturnus vulgaris
Coordinates: [18.638766537776622, 57.68275759304657]



# Swedish species observations invasive

https://sosgeo.artdata.slu.se/geoserver/web/wicket/bookmarkable/org.geoserver.web.demo.MapPreviewPage?3&filter=false


In [17]:
import pandas as pd

# Replace 'your_file_path.csv' with the actual path to your CSV file
file_path = 'SpeciesObservationsInvasive.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

In [18]:

# Display the first few rows of the DataFrame to inspect the data
df.head(1)


,FID,occurrenceId,url,isPresentObservation,dataProviderId,datasetName,startDate,endDate,modified,decimalLatitude,...,project1Category,project1Url,project1Values,project2Id,project2Name,project2Category,project2Url,project2Values,point,pointLocation
0,SpeciesObservationsInvasive.iO_EE44BcnWOXMpyz808,urn:lsid:herbarium.emg.umu.se:observation:UPS*...,NaN,True,7,Virtual Herbarium,1852-08-12T23:00:00,1852-08-12T23:00:00,NaN,57.79459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (18.79638 57.79459),POINT (18.79638 57.79459)


In [21]:
# Find unique values of the column 'isInvasiveInSweden'
unique_values = df['province'].unique()

# Display the unique values
print(unique_values)


['Gotland' 'Halland' 'Bohuslän' 'Uppland' 'Västerbotten' 'Södermanland'
 'Jämtland' 'Pite lappmark' 'Gästrikland' 'Småland' 'Västmanland' 'Skåne'
 'Värmland' 'Blekinge' 'Öland']


In [22]:
# Find unique values of the column 'scientificName'
unique_species = df['scientificName'].unique()

# Display the unique species names
print(unique_species)


['Linum bienne' 'Dasya baillouviana' 'Magallana gigas' 'Sargassum muticum'
 'Oncorhynchus mykiss' 'Gilia achilleifolia' 'Dasysiphonia japonica'
 'Amphibalanus improvisus' 'Pinus contorta' 'Brugmansia suaveolens'
 'Gracilaria vermiculophylla' 'Rhizoctonia solani'
 'Pacifastacus leniusculus' 'Coreopsis tinctoria' 'Phyteuma nigrum'
 'Mentha suaveolens' 'Acer pseudoplatanus' 'Phyteuma spicatum'
 'Potamopyrgus antipodarum']


Among the list provided, the following entries are plants:

- Linum bienne
- Gilia achilleifolia
- Pinus contorta
- Brugmansia suaveolens
- Coreopsis tinctoria
- Phyteuma nigrum
- Mentha suaveolens
- Acer pseudoplatanus
- Phyteuma spicatum

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   FID                            50 non-null     object 
 1   occurrenceId                   50 non-null     object 
 2   url                            7 non-null      object 
 3   isPresentObservation           50 non-null     bool   
 4   dataProviderId                 50 non-null     int64  
 5   datasetName                    50 non-null     object 
 6   startDate                      50 non-null     object 
 7   endDate                        50 non-null     object 
 8   modified                       10 non-null     object 
 9   decimalLatitude                50 non-null     float64
 10  decimalLongitude               50 non-null     float64
 11  coordinateUncertaintyInMeters  50 non-null     int64  
 12  locality                       10 non-null     objec